In [2]:
import math
import numpy as np
import pandas as pd
import altair as alt

In [19]:
data = {
    "NBD ON": [0.51, 0.36],
    "NBD LW": [0.46, 0.28],
    "NDB PIR": [0.42, 0.29],
    "SM PIR": [0.83, 0.86],
    "SM SP": [0.91, 0.92],
    "LFX": [41, 42],
}
data_bounds = {
    "NBD ON": [1.0, 0.0],  # lower is better
    "NBD LW": [1.0, 0.0],
    "NDB PIR": [1.0, 0.0],
    "SM PIR": [0.0, 1.0],  # higher is better
    "SM SP": [0.0, 1.0],
    "LFX": [200, 0],
}


def norm(v, v_min, v_max):
    # print(v, v_min, v_max)
    if v_min < v_max:
        return [(vi - v_min) / (v_max - v_min) for vi in v]
    return [1 - (vi) / (v_min - v_max) for vi in v]


data_normalized = {
    k: norm(v, v_min, v_max)
    for (k, v), (_, (v_min, v_max)) in zip(data.items(), data_bounds.items())
}

data_normalized

{'NBD ON': [0.49, 0.64],
 'NBD LW': [0.54, 0.72],
 'NDB PIR': [0.5800000000000001, 0.71],
 'SM PIR': [0.83, 0.86],
 'SM SP': [0.91, 0.92],
 'LFX': [0.795, 0.79]}

In [20]:
nb = len(list(data.keys()))
nb

6

In [21]:
angle = np.deg2rad([i * 360 / nb for i in range(0, nb)])
np.rad2deg(angle)

array([  0.,  60., 120., 180., 240., 300.])

In [22]:
radius = 1
steps = 5


def build_circle(radius):
    circle = [
        (
            radius * math.cos(i),
            radius * math.sin(i),
        )
        for i in angle
    ]
    circle.append(circle[0])
    return circle


def build_radial(radius):
    edges = [
        (
            radius * math.cos(i),
            radius * math.sin(i),
        )
        for i in angle
    ]
    radial = [[(0.0, 0.0), (p[0], p[1])] for p in edges]
    return [p for r in radial for p in r]


def build_legend(radius):
    edges = [
        (
            radius * math.cos(i),
            radius * math.sin(i),
        )
        for i in angle
    ]
    return edges


coordinates0 = [
    (v[0] * radius * math.cos(angle[i]), v[0] * radius * math.sin(angle[i]))
    for i, (k, v) in enumerate(data_normalized.items())
]
coordinates0.append(coordinates0[0])
coordinates1 = [
    (v[1] * radius * math.cos(angle[i]), v[1] * radius * math.sin(angle[i]))
    for i, (k, v) in enumerate(data_normalized.items())
]
coordinates1.append(coordinates1[0])
coordinates0, coordinates1

([(0.49, 0.0),
  (0.2700000000000001, 0.4676537180435969),
  (-0.2899999999999999, 0.5022947341949745),
  (-0.83, 1.0164568432923031e-16),
  (-0.4550000000000004, -0.788083117443839),
  (0.39750000000000013, -0.6884901960086287),
  (0.49, 0.0)],
 [(0.64, 0.0),
  (0.36000000000000004, 0.6235382907247957),
  (-0.3549999999999998, 0.6148780366869514),
  (-0.86, 1.0531962472667238e-16),
  (-0.4600000000000004, -0.7967433714816834),
  (0.39500000000000013, -0.6841600689897065),
  (0.64, 0.0)])

In [23]:
def project(data, color, sz=1, sd=[1]):
    df = pd.DataFrame(
        {
            "x": [x for x, y in data],
            "y": [y for x, y in data],
        }
    )
    return (
        alt.Chart(df)
        .mark_line()
        .encode(
            alt.Latitude("x:Q"),
            alt.Longitude("y:Q"),
            color=alt.value(color),
            size=alt.value(sz),
            strokeDash=alt.value(sd),
        )
        .project("identity")
        # .project(type="azimuthalEquidistant", rotate=[0, 0, 90])
    )


def projectEQ(data, dataEQ, sz=1, sd=[1]):
    df = pd.DataFrame(
        {
            "x": [x for x, y in data] + [x for x, y in dataEQ],
            "y": [y for x, y in data] + [y for x, y in dataEQ],
            "Score": ["no EQ" for i in range(0, len(data))]
            + ["auto EQ" for i in range(0, len(data))],
        }
    )
    return (
        alt.Chart(df)
        .mark_line()
        .encode(
            alt.Latitude("x:Q"),
            alt.Longitude("y:Q"),
            color=alt.Color("Score:N"),
            size=alt.value(3),
        )
        .project("identity")
        # .project(type="azimuthalEquidistant", rotate=[0, 0, 90])
    )


def text(coord, data):
    df = pd.DataFrame({"x": [x for x, y in coord], "y": [y for x, y in coord], "t": data.keys()})
    return (
        alt.Chart(df)
        .mark_text()
        .encode(
            alt.Latitude("x:Q"),
            alt.Longitude("y:Q"),
            text="t:O",
            size=alt.value(14),
        )
        .project("identity")
        # .project(type="azimuthalEquidistant", rotate=[0, 0, 90])
    )


ci = (
    project(build_circle(radius), "black", sd=[1, 3])
    + project(build_circle(radius * 0.75), "black", sd=[1, 3])
    + project(build_circle(radius * 0.50), "black", sd=[1, 3])
    + project(build_circle(radius * 0.25), "black", sd=[1.3])
)
cl = projectEQ(coordinates0, coordinates1)
r = project(build_radial(radius * 1.1), "black", sd=[1])
t = text(build_legend(radius * 1.3), data_normalized)
v1 = ci + r + cl + t

In [24]:
from math import sqrt, atan, sin, cos

z_colors = [
    "#5c77a5",
    "#dc842a",
    "#c85857",
    "#89b5b1",
    "#71a152",
    "#bab0ac",
    "#e15759",
    "#b07aa1",
    "#76b7b2",
    "#ff9da7",
]
geojson = {}
geojson["type"] = "FeatureCollection"
geojson["features"] = []

for i, coor in enumerate((coordinates0, coordinates1)):
    geojson["features"].append(
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[[x, y] for x, y in coor] + [[coor[0][0], coor[0][1]]]],
            },
            "properties": {
                "z_low": i,
                "z_high": i + 1,
                "stroke": "#000000",
                "stroke-opacity": 0.2,
                "stroke-width": 0,
                "fill-opacity": 0.2,
                "fill": z_colors[i % len(z_colors)],
            },
        }
    )
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.49, 0.0],
      [0.2700000000000001, 0.4676537180435969],
      [-0.2899999999999999, 0.5022947341949745],
      [-0.83, 1.0164568432923031e-16],
      [-0.4550000000000004, -0.788083117443839],
      [0.39750000000000013, -0.6884901960086287],
      [0.49, 0.0],
      [0.49, 0.0]]]},
   'properties': {'z_low': 0,
    'z_high': 1,
    'stroke': '#000000',
    'stroke-opacity': 0.2,
    'stroke-width': 0,
    'fill-opacity': 0.2,
    'fill': '#5c77a5'}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.64, 0.0],
      [0.36000000000000004, 0.6235382907247957],
      [-0.3549999999999998, 0.6148780366869514],
      [-0.86, 1.0531962472667238e-16],
      [-0.4600000000000004, -0.7967433714816834],
      [0.39500000000000013, -0.6841600689897065],
      [0.64, 0.0],
      [0.64, 0.0]]]},
   'properties': {'z_low': 1,
    'z_high': 2,
   

In [71]:
cs = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape()
    .encode(
        color=alt.Color(
            "properties.z_low:O",
            scale=alt.Scale(domain=[0, 1], range=["red", "green"]),
            legend=alt.Legend(title="Scores"),
        ),
        opacity=alt.value(0.2),
    )
    .project(
        type="identity",
        center=[0, 0],
        scale=200,
        reflectY=True,
        rotate=[0, 0, 0],
        translate=[100, 100],
    )
)

In [72]:
v2 = (ci + r + cs + t).properties(width=400)
(v1 | v2).resolve_scale(color="independent")

alt.HConcatChart(...)

In [76]:
geojson_experiment = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[[0, 0], [0, 1.2], [1.2, 1], [1, 0], [0, 0]]],
            },
            "properties": {
                "z_low": 0,
                "z_high": 1,
                "stroke": "#000000",
                "stroke-opacity": 0.2,
                "stroke-width": 0,
                "fill-opacity": 0.2,
                "fill": "#5c77a5",
            },
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[[0, 0], [-1, 0.5], [-2, 0], [-2, -2], [-0.2, -1.2], [0, 0]]],
            },
            "properties": {
                "z_low": 1,
                "z_high": 2,
                "stroke": "#000000",
                "stroke-opacity": 0.2,
                "stroke-width": 0,
                "fill-opacity": 0.2,
                "fill": "#dc842a",
            },
        },
    ],
}

scale = 100
square = (
    alt.Chart(alt.Data(values=geojson_experiment["features"]))
    .mark_geoshape(
        clip=False,
    )
    .encode(
        color=alt.Color(
            "properties.z_low:O",
            scale=alt.Scale(domain=[0, 1], range=["red", "green"]),
            legend=alt.Legend(title="Scores"),
        ),
        opacity=alt.value(0.2),
    )
    .project(type="identity", scale=scale, reflectX=False, reflectY=True)
)

empty = pd.DataFrame({"x": [-2, -2, -0.2, 0, 0, 1, 1.2, 2], "y": [-2, 0, -1.2, 0, 1.2, 0, 1, 2]})

axis = (
    alt.Chart(empty)
    .mark_point(clip=True)
    .encode(x=alt.X("x", scale=alt.Scale(type="linear")), y="y")
    .project(type="identity")
)

x_scale = scale * 4
y_scale = scale * 4
(square + axis).properties(width=x_scale, height=y_scale)

alt.LayerChart(...)

In [119]:
def t(x):
    return math.log10(x)


def r(x):
    return math.pi * x / 180


geojson_experiment = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [t(20), r(-180)],
                        [t(20000), r(-180)],
                        [t(20000), r(180)],
                        [t(20), r(180)],
                        [t(20), r(-180)],
                    ]
                ],
            },
            "properties": {
                "z_low": 0,
                "z_high": 1,
                "stroke": "#000000",
                "stroke-opacity": 0.2,
                "stroke-width": 0,
                "fill-opacity": 0.2,
                "fill": "#5c77a5",
            },
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [t(1000), r(-120)],
                        [t(10000), r(-120)],
                        [t(10000), r(120)],
                        [t(1000), r(120)],
                        [t(1000), r(-120)],
                    ]
                ],
            },
            "properties": {
                "z_low": 1,
                "z_high": 2,
                "stroke": "#000000",
                "stroke-opacity": 0.2,
                "stroke-width": 0,
                "fill-opacity": 0.2,
                "fill": "#dc842a",
            },
        },
    ],
}

scale = 40
square = (
    alt.Chart(alt.Data(values=geojson_experiment["features"]))
    .mark_geoshape(
        clip=False,
    )
    .encode(
        color=alt.Color(
            "properties.z_low:O",
            scale=alt.Scale(domain=[0, 1], range=["red", "green"]),
            legend=alt.Legend(title="Scores"),
        ),
        opacity=alt.value(0.2),
    )
    .project(type="identity", scale=200)
)

empty = pd.DataFrame(
    {"x": [t(20), t(1000), t(10000), t(20000)], "y": [r(-180), r(-120), r(120), r(180)]}
)
print(empty)
axis = (
    alt.Chart(empty)
    .mark_point(clip=True)
    .encode(x=alt.X("x", scale=alt.Scale(type="linear")), y="y")
)

x_scale = scale * 4
y_scale = scale * 4
(square + axis).properties(width=1200, height=1200)

         x         y
0  1.30103 -3.141593
1  3.00000 -2.094395
2  4.00000  2.094395
3  4.30103  3.141593


alt.LayerChart(...)